# **Guide d'utilisation pour le package backtester_poo_272_mcd**

Ce fichier Jupyter Notebook illustre l'utilisation du package backtester_poo_272_mcd pour effectuer des backtests. Le guide couvre l'importation des modules, le chargement des données, la configuration des stratégies, et l'analyse des résultats.

## **Importation des modules**

In [1]:
# Importation des modules principaux
from backtester_poo_272_mcd.tools import InputType, FrequencyType, Index, Benchmark
from backtester_poo_272_mcd.strategies import *
from backtester_poo_272_mcd import Backtester, DataInput, Results
import pandas as pd

## **Chargement des données**

Pour charger les données, vous disposez de plusieurs options flexibles selon vos besoins et vos sources de données :

1. **API Yahoo Finance** : Permet de récupérer des données financières historiques (actions, indices, etc.).
2. **API Binance** : Fournit des données pour les cryptomonnaies.
3. **Dataset Pandas** : Idéal si vous travaillez avec un DataFrame chargé directement dans votre environnement.
4. **Fichier CSV** : Pratique pour utiliser des données locales stockées dans un fichier CSV.

Le type d'importation des données est défini par l'enum `InputType`, et il est obligatoire de spécifier une fréquence des données via `FrequencyType`. Il est également possible de spécifier un benchmark à partir des benchmarks préetablis dans l'enum `Benchmark`.

Pour effectuer ce chargement, nous utilisons la classe `DataInput`, qui permet l'acquisition et de la mise en forme des données.

---

#### **1. Données Actions avec Yahoo Finance** (tickers ou indice) 

Pour importer des données d'actions via l'API Yahoo Finance, configurez les paramètres suivants :  
- **Liste des tickers** (ex. : `['AAPL', 'MSFT', 'GOOGL']`).  
- **Date de début** et **Date de fin** (format : `YYYY-MM-DD`).  
- **Fréquence des données** (ex. : Journalier, Hebdomadaire, etc.).  
- **Benchmark** (ex. : `Benchmark.SP500`).  voir liste des benchmarks disponibles sous le dossier tools

⚠️ **Note** : Assurez-vous que les tickers sont au format Yahoo Finance (différent du format Bloomberg par exemple).

In [2]:
data = DataInput(
    data_type=InputType.EQUITY,
    tickers=['AAPL', 'MSFT', 'GOOGL'],  # Liste des tickers
    start_date='2010-01-01',            # Date de début
    end_date='2024-01-01',              # Date de fin
    frequency=FrequencyType.WEEKLY,     # Fréquence des données (Journalier, Hebdomadaire, etc.)
    benchmark=Benchmark.SP500          # Benchmark à utiliser
)

Vous pouvez également utiliser les tickers des composants d'un indice boursier directement dans vos backtests en vous appuyant sur l'enum `Index`. Cette fonctionnalité permet de récupérer automatiquement les tickers associés à un indice particulier (par exemple, CAC40, S&P500, etc.) dans le format compatible avec Yahoo Finance.

**Remarque importante** : Seuls quelques indices sont actuellement disponibles, car leurs compositions ont été recensées manuellement pour correspondre aux formats requis par Yahoo Finance. L'ajout d'autres indices nécessite un travail supplémentaire de recherche et de mise en forme des données.

In [4]:
data = DataInput(
    data_type=InputType.FROM_INDEX_COMPOSITION,
    index=Index.CAC40,                   # Spécifier l'indice
    start_date='2015-01-01',             # Date de début
    end_date='2024-01-01',               # Date de fin
    frequency=FrequencyType.WEEKLY,       # Fréquence des données
    benchmark=Benchmark.CAC40            # Benchmark correspondant
)

The ticker URW.PA does not have a value in 2015-01-01


#### **2. Données Cryptomonnaies avec Binance**

Notre package permet d'importer des données historiques de cryptomonnaies via l'API Binance.  
Pour cela, il suffit de fournir :  
- **Date de début** et **Date de fin** (format : `YYYY-MM-DD`).  
- **Tickers** Binance (ex. : `['BTCUSDT', 'ETHUSDT']`). 
- **Fréquence des données** (ex. : Journalier, Hebdomadaire, etc.).  
- **Benchmark** (ex. : `Benchmark.BTC`), voir liste des benchmarks disponibles sous le dossier tools.    

⚠️ **Note** : Vérifiez la validité des tickers selon Binance.

In [5]:
data = DataInput(
    data_type=InputType.CRYPTO,
    tickers=['BTCUSDT', 'ETHUSDT', 'SOLUSDT'],   # Paires de cryptomonnaies
    start_date='2020-01-01',         # Date de début
    end_date='2024-01-01',           # Date de fin
    frequency=FrequencyType.DAILY,   # Fréquence des données (Horaire, Journalier, etc.)
    benchmark=Benchmark.BTC         # Benchmark à utiliser
)

#### **3. Importation par DataFrame Python**

Il est également possible d'importer des données depuis un **DataFrame Python**. Pour ce faire, utilisez le paramètre `data_type=InputType.FROM_DATAFRAME` et fournissez un DataFrame `df` contenant vos données. Vous devrez spécifier la fréquence des données (par exemple : Journalier) et le **benchmark** souhaité (par exemple : `Benchmark.CAC40`). De plus, il est nécessaire de définir une colonne `Date`

Si vous utilisez un benchmark, il est nécessaire de définir une **date de début** et une **date de fin**, car le benchmark sera récupéré via l'API Yahoo Finance en fonction de ces dates. La date de début et de fin doit correspondre à celle du dataframe


In [19]:
data = DataInput(
    data_type=InputType.EQUITY,
    tickers=['AAPL', 'MSFT', 'GOOGL'],  # Liste des tickers
    start_date='2010-01-01',            # Date de début
    end_date='2023-12-22',              # Date de fin
    frequency=FrequencyType.WEEKLY,     # Fréquence des données (Journalier, Hebdomadaire, etc.)
)

df = data.df_prices # On recupere le dataframe des prix
data = DataInput(data_type=InputType.FROM_DATAFRAME,
                custom_df=df,
                start_date='2010-01-01',            # Date de début
                end_date='2023-12-22',  
                benchmark=Benchmark.SP500,
                frequency=FrequencyType.WEEKLY)


### **4. Importation depuis un Fichier CSV**

Il est également possible d'importer des données depuis un fichier exel ou CSV. Pour cela, vous pouvez utiliser un chemin local.

Utilisez le paramètre `data_type=InputType.FROM_FILE` et fournissez le chemin du fichier. Assurez-vous que votre fichier excel ou CSV contient une colonne `Date` avec des dates formatées correctement.

Si un benchmark est utilisé, vous devez définir une **date de début** et une **date de fin**, car le benchmark sera récupéré via l'API Yahoo Finance en fonction de ces dates. Ces dates doivent correspondre à la période des données dans le fichier CSV.

In [29]:
data = DataInput(
    data_type=InputType.EQUITY,
    tickers=['AAPL', 'MSFT', 'GOOGL'],  # Liste des tickers
    start_date='2010-01-01',            # Date de début
    end_date='2023-12-22',              # Date de fin
    frequency=FrequencyType.WEEKLY,     # Fréquence des données (Journalier, Hebdomadaire, etc.)
)

df = data.df_prices # On recupere le dataframe des prix
df.to_excel("custom.xlsx", index = False) # Sauvegarde du csv

In [33]:
path_excel = "custom.xlsx"
data = DataInput( # Lecture
    data_type=InputType.FROM_FILE,
    file_path=path_excel,  # Lien vers le fichier CSV ou excel
    frequency=FrequencyType.WEEKLY,              # Fréquence des données
    benchmark=Benchmark.SP500,                  # Benchmark à utiliser
    start_date='2010-01-01',            # Date de début
    end_date='2023-12-22',                       # Date de fin
)

## Initialisation du backtester

Le **Backtester** est une classe générique conçue pour tester des stratégies d'investissement à partir des données historiques des prix des actifs. Son initialisation repose principalement sur l'objet `DataInput`, qui contient les données nécessaires à l'exécution du backtest, notamment les prix historiques des actifs sélectionnés et ceux du benchmark si nécessaire.

Voici un exemple avec 5 actifs.

In [34]:
data = DataInput(data_type=InputType.EQUITY,
                        tickers=['GLE.PA', 'OR.PA','MC.PA','VIV.PA','TTE.PA'],
                        start_date='2015-10-01',
                        end_date='2024-10-01',
                        frequency=FrequencyType.WEEKLY,
                        benchmark=Benchmark.CAC40)

backtest = Backtester(data_input=data)

## Initialisation des stratégies

Les **stratégies** constituent le cœur du backtester et sont conçues pour encapsuler les différentes logiques d'investissement utilisées lors des simulations. Elles sont basées sur une classe abstraite et peuvent être définies de manière flexible, soit par héritage, soit via un décorateur pour des stratégies plus simples.

Chaque stratégie nécessite plusieurs arguments : 
- **`rebalance_frequency`** *(FrequencyType)* : Spécifie la fréquence de rééquilibrage de la stratégie (par exemple, mensuel ou hebdomadaire). La valeur par défaut est mensuelle.
- **`lookback_period`** *(float)* : Définit la période historique (en années) utilisée pour calculer les indicateurs ou prendre des décisions. La valeur par défaut est d'un an.

Certaines stratégies, dites **Long/Short**, nécessitent un paramètre additionnel :
- **`is_LS_strategy`** *(Optional[bool])* : Indique si la stratégie est une stratégie Long/Short. Sa valeur par défaut est `False`.

Voici un aperçu de certaines stratégies déjà disponibles (liste non exhaustive) :

- **Momentum Strategy** : Basée sur l'élan des performances passées.
- **Mean Reverting Strategy** : Cherche à profiter du retour à la moyenne.
- **Trend Following Strategy** : Suit les tendances établies sur le marché.
- **Low Volatility Strategy** : Privilégie les actifs à faible volatilité.
- **Optimal Sharpe Strategy** : Optimise les pondérations pour maximiser le ratio de Sharpe.

In [36]:
from backtester_poo_272_mcd.strategies import * # Importation des stratégies
'''Initialisation des stratégies '''
strategy_momentum = MomentumStrategy(rebalance_frequency=FrequencyType.MONTHLY, lookback_period=1, is_LS_strategy=True)
strategy_mr = MeanRevertingStrategy(rebalance_frequency=FrequencyType.MONTHLY, lookback_period=1, is_LS_strategy=False)
strategy_tf = TrendFollowingStrategy(rebalance_frequency=FrequencyType.MONTHLY, short_window_period=10, long_window_period=50, is_LS_strategy=False)
strategy_low = LowVolatilityStrategy(rebalance_frequency=FrequencyType.MONTHLY, lookback_period=1)

strategy_mkw = OptimalSharpeStrategy(rebalance_frequency=FrequencyType.MONTHLY, lookback_period=1)
stategy_mkw_vol = OptimalLowVolatilityStrategy(rebalance_frequency=FrequencyType.MONTHLY, lookback_period=1)

strategy_random = RandomFluctuationStrategy(rebalance_frequency=FrequencyType.MONTHLY, lookback_period=0)
strategy_eqw = EqualWeightStrategy(rebalance_frequency=FrequencyType.MONTHLY, lookback_period=0)

### **Exécution du Backtesting**

La méthode `run` du backtester exécute une simulation complète de stratégie d'investissement sur une période donnée, tout en comparant les résultats à un benchmark si ce dernier est sélectionné.

Voici une explication détaillée du fonctionnement du processus :

#### 1. **Vérification des Entrées**
Avant de lancer la simulation, la méthode effectue plusieurs vérifications sur les paramètres d’entrée :
- **Montant initial** : Vérifie que le montant initial du portefeuille est un nombre positif.
- **Frais de transaction** : Les frais de transaction doivent être compris entre 0 et 1.
- **Fréquence de rééquilibrage** : La fréquence de rééquilibrage de la stratégie ne peut pas être plus fréquente que la fréquence des données de prix.
- **Période historique suffisante** : S'assure qu'il y a suffisamment de données historiques pour calculer les indicateurs nécessaires au backtest.

#### 2. **Préparation des Données**
La méthode prépare les dates de rééquilibrage et ajuste la période d'historique nécessaire pour la stratégie. Ensuite, elle initialise les valeurs du portefeuille, les rendements et les prix des actifs, ainsi que les poids du portefeuille à l'instant initial.

#### 3. **Simulation du Backtest**
La méthode boucle sur chaque période (chaque jour de trading) et procède aux étapes suivantes :
- **Calcul des rendements du portefeuille** : Les rendements du portefeuille sont calculés en fonction des poids actuels et des rendements journaliers des actifs.
- **Rééquilibrage** : Si c'est le moment de rééquilibrer, la stratégie fournit les nouveaux poids, qui sont appliqués au portefeuille. Les coûts de transaction sont également pris en compte lors du rééquilibrage.
- **Application du "drift"** : Si aucune opération de rééquilibrage n'a lieu, les poids dérivent par effet de marché.
- **Calcul du portefeuille benchmark** : Si un benchmark est défini, sa valeur est mise à jour à chaque étape pour être comparée à celle de la stratégie.

#### 4. **Stockage des Résultats**
Les valeurs du portefeuille, les poids des actifs et les valeurs du benchmark sont stockés au fur et à mesure de la simulation. Cela permet de suivre l'évolution de la stratégie au fil du temps.

#### 5. **Génération des Résultats**
Une fois la simulation terminée, la méthode génère un objet `Results` qui contient :
- La valeur du portefeuille de la stratégie au fil du temps.
- Les poids des actifs dans le portefeuille à chaque rééquilibrage.
- Les statistiques de performance (rendement, volatilité, etc.).
- Les graphiques comparatifs avec le benchmark si sélectionné.

#### 6. **Retour des Résultats**
La méthode retourne l'objet `Results` contenant toutes les statistiques, les graphiques et les comparaisons avec le benchmark si disponible.

---

### **Conclusion**
En résumé, la méthode `run` exécute un backtest détaillé de la stratégie choisie, prenant en compte les rééquilibrages, les frais de transaction et l’évolution du portefeuille. Les résultats sont ensuite retournés sous la forme d’un objet contenant des informations statistiques et visuelles pour évaluer la performance de la stratégie par rapport à un benchmark.


In [37]:
'''Initialisation des données'''
data = DataInput(data_type=InputType.EQUITY,
                        tickers=['GLE.PA', 'OR.PA','MC.PA','VIV.PA','TTE.PA'],
                        start_date='2015-10-01',
                        end_date='2024-10-01',
                        frequency=FrequencyType.WEEKLY,
                        benchmark=Benchmark.CAC40)
'''Initialisation du backtest'''
backtest = Backtester(data_input=data)
'''Initialisation de la stratégie'''
strategy_eqw = EqualWeightStrategy(rebalance_frequency=FrequencyType.MONTHLY, lookback_period=0)
'''Run du backtest'''
results_eqw = backtest.run(strategy=strategy_eqw, initial_amount=1000.0, fees=0.0001)

Running Backtesting EqualWeightStrategy: 100%|██████████| 470/470 [00:00<00:00, 22813.86it/s]


Backtester.output took 0.2210 sec to execute
Backtester.run took 0.2744 sec to execute


Le module de **visualisation des résultats** d'un backtest génère et affiche des statistiques et des graphiques permettant d'analyser la performance d'une stratégie d'investissement. Voici un aperçu des principales fonctionnalités de la classe `Results` qui permet de gérer et d'afficher les résultats du backtest :

---

#### **1. Statistiques Calculées** :
- **Retour Total** : Le rendement total sur la période.
- **Retour Annualisé** : Le rendement annualisé, tenant compte de la fréquence des données.
- **Volatilité Annualisée** : La volatilité annualisée du portefeuille.
- **Ratio de Sharpe** : Mesure du rendement ajusté en fonction du risque.
- **Ratio de Sortino** : Semblable au ratio de Sharpe, mais utilisant la volatilité des rendements négatifs.
- **Drawdowns** : La perte maximale par rapport au sommet historique du portefeuille.
- **VaR (Value at Risk)** : La perte maximale attendue à un niveau de confiance donné.
- **CVaR (Conditional VaR)** : La moyenne des pertes excédant le VaR.

---

#### **2. Graphiques Générés** :
- **Graphique de la performance de la stratégie** : Montre l'évolution de la valeur du portefeuille sur le temps.
- **Graphique des Drawdowns** : Affiche les drawdowns (pertes maximales) de la stratégie au fil du temps.
- **Graphique des poids du portefeuille** : Montre l'évolution des poids des actifs dans le portefeuille au fil du temps.

---

#### **3. Fonctionnalités de Comparaison** :
- Il est possible de comparer les résultats de plusieurs stratégies (y compris un benchmark) en combinant les graphiques de valeurs, de drawdowns et les poids de portefeuille.
- La méthode `compare_results` combine les résultats de plusieurs stratégies dans un seul objet `Results`, permettant de visualiser et de comparer les performances sur un même graphique.

---

#### **4. Création et Mise à Jour des Graphiques** :
- Les graphiques sont générés avec **Plotly**, une bibliothèque interactive de visualisation.
- Les graphiques de valeurs et de drawdowns sont dynamiques, permettant une analyse détaillée de la performance des stratégies.

---

In [38]:
display(results_eqw.df_statistics.head(10))
results_eqw.ptf_value_plot.show()
results_eqw.ptf_weights_plot.show()
results_eqw.ptf_drawdown_plot.show()

,Metrics,EqualWeightStrategy,Benchmark
0,Total Return,186.59%,69.86%
1,Annualized Return,12.33%,6.02%
2,Volatility,20.54%,19.00%
3,Sharpe Ratio,0.50,0.21
4,Sortino Ratio,0.65,0.26
5,Max Drawdown,-34.52%,-33.63%
6,VaR 95%,-4.21%,-3.91%
7,CVaR 95%,-6.49%,-6.18%


### **Comparaison entre plusieurs stratégies**

Nous allons comparer deux stratégies d'investissement sur les cryptomonnaies en utilisant les résultats générés par la classe `Results` :

1. **Stratégie Low Vol Traditionnelle** : Une stratégie basée sur l'investissement dans les actifs ayant la volatilité la plus faible. Cette approche est simple et utilise des critères historiques de volatilité pour sélectionner les actifs du portefeuille.

2. **Stratégie Low Vol Optimale de Markowitz** : Une stratégie d'allocation de portefeuille basée sur la théorie moderne du portefeuille de Markowitz. Elle cherche à minimiser la volatilité tout en optimisant le rendement attendu, en tenant compte des corrélations entre les actifs.

Nous allons pouvoir comparer les performances de ces deux stratégies grâce à la fonction `compare_results` en termes de **rendement total**, **volatilité**, **ratio de Sharpe**, **drawdowns**, et autres métriques, tout en visualisant les différences de poids dans le portefeuille et les impacts sur les performances.




In [39]:
'''Initialisation des données'''
data = DataInput(
    data_type=InputType.CRYPTO,
    tickers=['BTCUSDT', 'ETHUSDT', 'SOLUSDT'],   # Paires de cryptomonnaies
    start_date='2020-01-01',         # Date de début
    end_date='2024-01-01',           # Date de fin
    frequency=FrequencyType.DAILY,   # Fréquence des données (Horaire, Journalier, etc.)
    benchmark=Benchmark.BTC         # Benchmark à utiliser
)
'''Initialisation du backtest'''
backtest = Backtester(data_input=data)
'''Initialisation des stratégies'''
strategy_low = LowVolatilityStrategy(rebalance_frequency=FrequencyType.MONTHLY, lookback_period=1)
stategy_mkw_vol = OptimalLowVolatilityStrategy(rebalance_frequency=FrequencyType.MONTHLY, lookback_period=1)
'''Run du backtest'''
results_low = backtest.run(strategy=strategy_low, initial_amount=1000.0, fees=0.0)
results_mkw = backtest.run(strategy=stategy_mkw_vol, initial_amount=1000.0, fees=0.0)
combined_results = Results.compare_results([results_low,results_mkw])
'''Visualisation des résultats'''
print(combined_results.df_statistics.head(10))
combined_results.ptf_value_plot.show()
combined_results.ptf_drawdown_plot.show()
for plot in combined_results.ptf_weights_plot:
    plot.show()

Running Backtesting LowVolatilityStrategy: 100%|██████████| 1209/1209 [00:00<00:00, 33090.67it/s]


Backtester.output took 0.1161 sec to execute
Backtester.run took 0.1840 sec to execute


Running Backtesting OptimalLowVolatilityStrategy: 100%|██████████| 1209/1209 [00:01<00:00, 1104.94it/s]


Backtester.output took 0.1158 sec to execute
Backtester.run took 1.2139 sec to execute
             Metrics LowVolatilityStrategy OptimalLowVolatilityStrategy  \
0       Total Return               817.14%                      392.75%   
1  Annualized Return                58.65%                       39.40%   
2         Volatility                62.45%                       53.95%   
3       Sharpe Ratio                  0.91                         0.69   
4      Sortino Ratio                  1.25                         0.97   
5       Max Drawdown               -82.68%                      -76.90%   
6            VaR 95%                -5.99%                       -5.28%   
7           CVaR 95%                -8.96%                       -7.84%   

  Benchmark  
0   332.32%  
1    35.65%  
2    53.36%  
3      0.63  
4      0.89  
5   -76.63%  
6    -5.16%  
7    -7.73%  


Voici un autre exemple sur une stratégie momentum et TrendFollowing

In [41]:
'''Initialisation des données'''
data = DataInput(data_type=InputType.EQUITY,
                        tickers=['GLE.PA', 'OR.PA','MC.PA','VIV.PA','TTE.PA'],
                        start_date='2015-10-01',
                        end_date='2024-10-01',
                        frequency=FrequencyType.WEEKLY,
                        benchmark=Benchmark.CAC40)
'''Initialisation du backtest'''
backtest = Backtester(data_input=data)
'''Initialisation des stratégies'''
strategy_momentum = MomentumStrategy(rebalance_frequency=FrequencyType.MONTHLY, lookback_period=1, is_LS_strategy=False)
strategy_tf = TrendFollowingStrategy(rebalance_frequency=FrequencyType.MONTHLY, short_window_period=10, long_window_period=50, is_LS_strategy=False)
'''Run du backtest'''
results_momentum = backtest.run(strategy=strategy_momentum, initial_amount=1000.0, fees=0.0)
results_tf = backtest.run(strategy=strategy_tf, initial_amount=1000.0, fees=0.0)
combined_results = Results.compare_results([results_momentum,results_tf])
'''Visualisation des résultats'''
print(combined_results.df_statistics.head(10))
combined_results.ptf_value_plot.show()
combined_results.ptf_drawdown_plot.show()
for plot in combined_results.ptf_weights_plot:
    plot.show()

Running Backtesting MomentumStrategy: 100%|██████████| 418/418 [00:00<00:00, 5717.96it/s]


Backtester.output took 0.0717 sec to execute
Backtester.run took 0.1646 sec to execute


Running Backtesting TrendFollowingStrategy: 100%|██████████| 418/418 [00:00<00:00, 3031.50it/s]


Backtester.output took 0.0732 sec to execute
Backtester.run took 0.2295 sec to execute
             Metrics MomentumStrategy TrendFollowingStrategy Benchmark
0       Total Return          109.41%                115.98%    70.27%
1  Annualized Return            9.61%                 10.03%     6.83%
2         Volatility           20.53%                 19.54%    18.74%
3       Sharpe Ratio             0.37                   0.41      0.26
4      Sortino Ratio             0.46                   0.54      0.30
5       Max Drawdown          -31.32%                -52.13%   -33.63%
6            VaR 95%           -4.47%                 -4.26%    -3.83%
7           CVaR 95%           -6.84%                 -6.20%    -6.26%
